In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time
import re  #regular expression (from bs documentation)
import selenium

In [ ]:
#https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# URL of page to be scraped
url = 'https://www.everydayhealth.com/drugs/truvada/reviews'
#adding in splinter needed item
browser.visit(url)

In [ ]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser' 
#allows our code to parse through the items
soup = bs(html, 'html.parser')
#Retrieve all elements that contain organization information in an iterable list
#results = soup.find('div', class_='star-rating-print').findChild()
results = soup.body.find('div', class_='review-container row').findChildren()
print(results)

In [ ]:
#test prints
html = browser.html
# results1 = soup.find('div',id='reviews-target')
results = soup.body.find_all(class_='review-container row')
for r in results:
    whats = r.h3.text
    print (whats)


In [ ]:
#This only iterates through one page with the drug in the html already, now need to loop through all diff drug review pgs
brands = []
what_fors = []
reviews = []
dates_posted = []
star_ratings = []

#however when looking at the code h3 has a span within it that has text and that is the brand name
#try for reasons to take
for r in results:
#if needed can add an if statement to say if r element has a__ then: ...
#    brand_what = r.h3.text
#    brand_what_strip = brand_what.strip("Rated")
    brand = r.h3.span.text
#    print(brand)
    brands.append(brand)
    
    #for reason (not sure yet how to only get the reason will work on later, maybe use if str.contains?)
    #ok ?? using re (regex to remove what I dont want)https://www.regular-expressions.info/quickstart.html
    #or do in df
    whats = r.h3.text
    print (whats)
    what_fors.append(whats)

    #need review which is in a p tag
    review = r.p.text #its scraping the last one twice not sure why? ok not doing it in actual code...hmmmm
#    print(review)
    reviews.append(review)
    
    #need the date published, its in a span errrrr or really want the 
    #reference https://stackoverflow.com/questions/11205386/python-beautifulsoup-get-an-attribute-value-based-on-the-name-attribute
    date = r.find('span', class_='time')['content']
#    print (date)
    dates_posted.append(date)
    
    #jumping to the stars rating, will figure out date after
    rating = r.find('div', class_='star-rating-print').text.strip("Star")
#     print(rating)
    star_ratings.append(rating)
    
    
print(what_fors)

In [ ]:
drug_review_df = pd.DataFrame({'Name':brands,'Reason':what_fors,'Reviews':reviews,
                               'Date Posted':dates_posted,'Ratings':star_ratings})
drug_review_df

In [8]:
#https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


In [9]:
#46 brand name drugs
all_brand_hiv_drugs = ['Combivir', 'Emtriva', 'Epivir', 'Epizicom','Hivid','Trizivir',
    'Truvada','Videx','Viread','Zerit','Ziagen','Edurant','Intelence',
    'Rescriptor','Pifeltro','Sustiva','Viramune','Agenerase','Aptivus',
    'Crixivan','Invirase','Kaletra','Lexiva','Norvir','Prezista','Reyataz',
    'Viracept','Isentress','Tivicay','Vitekta','Atripla','Biktarvy',
    'Cimduo','Complera','Delstrigo','Dovato','Genvoya','Evotaz','Juluca',
    'Odefsey','Prezcobix','Symfi','Stribild','Symtuza','Triumeq','Descovy']

In [ ]:
# from urllib.request import Request, urlopen
    #trying this Nope?
#     req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
#     webpage = urlopen(req).read()
#     soup = bs(webpage, "html.parser")

In [7]:
#these will need to be w/n our loop & need a list of drugs (added above will need if statement if drug not found) 
drugs = ['atripla','truvada']
for d in drugs:
    # URL of page to be scraped
    url = f'https://www.everydayhealth.com/drugs/{d}/reviews'
    
    #adding in splinter needed item
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser' 
    soup = bs(html, 'html.parser')
    
    time.sleep(10)
    
    #Retrieve all elements that contain organization information in an iterable list
    if (soup.body.find(class_="error-404")):
        print("Drug Not Found")
    
    else:
        results = soup.body.find_all(class_='review-container row')
        print(results)
#ok this works now add to the info part of the scrape!

WebDriverException: Message: chrome not reachable
  (Session info: chrome=79.0.3945.130)


In [12]:
#Keep lists outside of loops
brands = []
what_fors = []
reviews = []
dates_posted = []
star_ratings = []

for d in all_brand_hiv_drugs:
    url = f'https://www.everydayhealth.com/drugs/{d}/reviews'
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser' 
    soup = bs(html, 'html.parser')
    #Retrieve all elements that contain organization information in an iterable list
    if (soup.body.find(class_="error-404")):
        print(f'{d}: Drug Not Found')
    
    elif (soup.body.find(class_="no-reviews")):
        print(f'{d}: No Reviews')
    
    else:
        results = soup.body.find_all(class_='review-container row')
#       print(results)

        for r in results:
            brand = d  #r.h3.span.text this would pull in generic name and some reviews dont include this
            brands.append(brand)
#             whats = r.h3.text
#             what_fors.append(whats)
#             review = r.p.text #its scraping the last one twice not sure why? ok not doing it in actual code...hmmmm
#             reviews.append(review)
#             date = r.find('span', class_='time')['content']
#             dates_posted.append(date)
#             rating = r.find('div', class_='star-rating-print').text
#             star_ratings.append(rating)
            print(brand)
    #going to fast? getting a too many redirects clear cookies
#     time.sleep(30)
print(brands)

#need to add loop through pages per drug if there are pages of reviews

Combivir: No Reviews
Emtriva: No Reviews
Epivir: No Reviews
EpizicomDrug Not Found
HividDrug Not Found
Trizivir: No Reviews
Truvada
Truvada
Truvada
Videx: No Reviews
Viread
Zerit: No Reviews
Ziagen: No Reviews
Edurant: No Reviews
Intelence: No Reviews
Rescriptor: No Reviews
Pifeltro: No Reviews
Sustiva: No Reviews
Viramune: No Reviews
AgeneraseDrug Not Found
Aptivus: No Reviews
Crixivan: No Reviews
Invirase: No Reviews
Kaletra: No Reviews
Lexiva: No Reviews
Norvir: No Reviews
Prezista: No Reviews
Reyataz: No Reviews
Viracept: No Reviews
Isentress
Tivicay
Vitekta: No Reviews
Atripla
Atripla
Atripla
Atripla
Atripla
Atripla
Atripla
Atripla
Atripla
Atripla
Atripla
Atripla
Biktarvy
Cimduo: No Reviews
Complera: No Reviews
Delstrigo
DovatoDrug Not Found
Genvoya
Genvoya
Genvoya
Genvoya
Evotaz: No Reviews
Juluca: No Reviews
OdefseyDrug Not Found
Prezcobix: No Reviews
Symfi: No Reviews
Stribild: No Reviews
Symtuza
Symtuza
Symtuza
Triumeq
Triumeq
Triumeq
Triumeq
Triumeq
DescovyDrug Not Found
['Tr

Drugs.com Scrape

In [ ]:
#https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

#Initiate browser w/ Splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# URL of page to be scraped (so too many to type with, without brand and generic)
url = 'https://www.drugs.com/comments/efavirenz-emtricitabine-tenofovir/atripla-for-hiv-infection.html'
#adding in splinter needed item
browser.visit(url)
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')
#Retrieve all elements that contain organization information in an iterable list
results = soup.find('ul', class_='ddc-paging')
lis = results.find_all('li')
print((len(lis))-1)

In [ ]:
hiv_drugs2 = ['emtricitabine-tenofovir/truvada-','efavirenz-emtricitabine-tenofovir/atripla-',
              'ritonavir/norvir-']
              #,'raltegravir/isentress-','darunavir/prezista-',
#               'cobicistat-elvitegravir-emtricitabine-tenofovir/stribild-',
#               'abacavir-dolutegravir-lamivudine/triumeq- ','dolutegravir/tivicay- ',
#               'emtricitabine-rilpivirine-tenofovir/complera-','abacavir-lamivudine/epzicom-',
#               'cobicistat-elvitegravir-emtricitabine-tenofovir-alafenamide/genvoya-',
#               'atazanavir/reyataz-','emtricitabine-tenofovir-alafenamide/descovy-',
#               'lamivudine/','tenofovir/viread-','lamivudine/epivir-','lopinavir-ritonavir/kaletra-',
#               'efavirenz/sustiva-','efavirenz-emtricitabine-tenofovir/','etravirine/intelence-','ritonavir/',
#               'emtricitabine-tenofovir/','tenofovir/','abacavir/','bictegravir-emtricitabine-tenofovir-alafenamide/biktarvy-',
#               'abacavir-lamivudine/','efavirenz/','raltegravir/','darunavir/','dolutegravir/','abacavir/ziagen-',
#               'nevirapine/viramune-','atazanavir/','lamivudine-zidovudine/combivir-','cobicistat-darunavir/prezcobix-',
#               'emtricitabine/emtriva-','fosamprenavir/lexiva-','emtricitabine-rilpivirine-tenofovir-alafenamide/odefsey-',
#               'zidovudine/','nevirapine/','abacavirlamivudine-zidovudine/trizivir-','indinavir/crixivan-','emtricitabine/',
#               'lamivudine/epivir-hbv','lopinavir-ritonavir/','stavudine/zerit-','abacavir-dolutegravir-lamivudine/',
#               'didanosine/','emtricitabine-rilpivirine-tenofovir/','etravirine/','indinavir/','lamivudine-zidovudine/',
#               'maraviroc/','stavudine/','didanosine/videx-','nelfinavir/viracept-','abacavir-lamivudine-zidovudine/',
#               'tipranavir/aptivus-','cobicistat-elvitegravir-emtricitabine-tenofovir-alafenamide/',
#               'atazanavir-cobicistat/evotaz-','raltegravir/isentress-hd-','dolutegravir-rilpivirine/juluca-',
#               'bictegravir-emtricitabine-tenofovir-alafenamide/ ','cobicistat/','delavirdine/','doravirine/',
#               'efavirenz-lamivudine-tenofovir/','emtricitabine-rilpivirine-tenofovir-alafenamide/',
#               'emtricitabine-tenofovir-alafenamide/','enfuvirtide/','rilpivirine/',
#               'cobicistat-darunavir-emtricitabine-tenofovir-alafenamide/symtuza-',
#               'cobicistat-darunavir-emtricitabine-tenofovir-alafenamide/','dolutegravir-rilpivirine/']

In [ ]:
brands = []
what_fors = []
reviews = []
dates_posted = []
star_ratings = []

for d in hiv_drugs2:
    # URL of page to be scraped (so too many to type with, without brand and generic)
    url = 'https://www.drugs.com/comments/{d}for-hiv-infection.html'
    #adding in splinter needed item
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser'
    soup = bs(html, 'html.parser')
    #Retrieve all elements that contain organization information in an iterable list
    results = soup.find('ul', class_='ddc-paging')
    lis = results.find_all('li')
    i = (len(lis))


    for x in range(1,i):
        #test prints
        html = browser.html
        results2 = soup.body.find('div', class_='contentBox')
        brand = results2.h2.text #why is this stripping the last letter?
        whats = "HIV"

        results = soup.body.find_all('div', class_='ddc-comment')
        for r in results:
            #Get the review text
            review = r.p.text 
            
            #Get the date published
            try:
                date = r.find('span',class_='comment-date').text
            except:
                date = 'n/a'
            
            #Get the rating divide by 2 because it is a rating of 1-10 and we only want a rating 1-5
            try:
                rating = r.find('div', class_='rating-score').text #will have to divide by 2
            except:
                rating = 'n/a'
    
    
            brands.append(brand)
            what_fors.append(whats)
            reviews.append(review)
            dates_posted.append(date)
            star_ratings.append(rating)   